#  Create a groundwater level map from borehole data and a DEM 
<hr >

## 1. Introduction

In this exercise you'll learn how to convert a Comma Separated Values file with borehole depth and coordinates to a PCRaster map. Next, we're going to interpolate the borehole depths to continuous raster using the Inverse Distance Weighting (IDW) method and the Thiessen method.

Let's first import PCRaster and set the path to the `data` folder where we have the input data for this exercise.

In [1]:
from pcraster import *
import os
print(os.getcwd())
os.chdir("./data")

/home/jacob/3.9.4/3.8.4/4.GIS_DEV/Spatial-Analysis-Studios/8.Geohydrological Applications/3.PCRasters/GroundwaterInterpolation


## 2. Convert a CSV file to PCRaster map format
Open the `boreholesdhxy.csv` file in the `data` folder in a text editor and check the contents.
Which columns are the coordinates and which column gives the depth of the borehole?

PCRaster has a few command line applications that are useful for these tasks. You can find these applications [here](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/applications.html). We will use <code>col2map</code> to convert the CSV file to a PCRaster map.

The syntax of <code>col2map</code> is <code>col2map [options] columnfile PCRresult</code>.
Here we'll use some options to indicate the output data type (Scalar) and order of the x, y and data column (v). We also need to indicate a clone map that has the raster properties (coordinate system, cell size, nr of rows/columns) of the desired output.

We can run command line commands from Python using <code>os.system(stringWithCommand)</code>:

In [2]:
command = "col2map -S -x 2 -y 3 -v 1 boreholesdxy.csv boreholedepth.map --clone clone.map"
os.system(command)

0

Read the `boreholedepth.map` file into the variable `BoreholeDepth`.

### Visualise `BoreholeDepth` with Aguila. 
* Since we run this Jupyter Notebook locally, we can use the `aguila` python operation. 
* The map will popup in the background, so you might need to look for it.

In [6]:
BoreholeDepth = readmap("boreholedepth.map")

In [7]:
aguila(BoreholeDepth)

## 3. Interpolate borehole depth with IDW
We can now interpolate the borehole depth at the pixels. We'll first apply the Inverse Distance Weighting method. This has been implemented as the [inversedistance operation](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_inversedistance.html) in PCRaster.

The syntax is: 
```Python
Result = inversedistance( mask, points, idp, radius, maxNr)
```

Where 
* <code>mask</code> is a mask of the extent that will be interpolated. In our case we'll use the clone map. 
* <code>points</code> is the scalar raster layer with pixels that you want to interpolate. In our case <code>BoreholeDepth</code>.
* <code>idp</code> is the exponent of the power function for the weights. An <code>idp</code> value of 2 is commonly used.
* <code>radius</code> indicates the distance to a cell within it has to take interpolation points into account. If it's set to 0 there's no limitation.
* <code>maxNr</code> limits the number of points used for the interpolation. If it's set to 0 there's no maximum.

Let's first read the clone map that we'll use as the mask.

In [8]:
Mask = readmap("clone.map")

Now we interpolate the borehole depths with IDW using the following line:

In [9]:
BoreholeDepthIDW = inversedistance( Mask, BoreholeDepth, 2, 0, 0)

Let's visualise the result with Aguila.

In [10]:
aguila(BoreholeDepthIDW)

## 4. Interpolate borehole depth with Thiessen polygons
<hr> 

Another interpolation method is using so called Thiessen polygons.

In PCRaster this involves three steps:

1. Assign a unique ID to each borehole
2. Assign to each pixel the closest borehole ID
3. Assign the borehole depth to the corresponding borehole IDs for all cells

Let's first assign a unique ID to each borehole using the <code>uniqueid</code> operation. The input needs to be a boolean raster.

To create a boolean raster of the boreholes we use the <code>defined</code> operation. This will give boolean True for all cells with a value and boolean False for all nodata cells.

In [11]:
Boreholes = defined(BoreholeDepth)

Check the result with Aguila.

In [13]:
aguila(Boreholes)

Now we assign a unique ID to each boolean True cell. 
- Note that we want the output to be nominal instead of scalar. 
- Therefore we wrap the <code>uniqueid</code> into the <code>nominal</code> operation.

In [14]:
BoreholeID = nominal(uniqueid(Boreholes))

In [15]:
aguila(BoreholeID)

Let's assign to each cell its closes borehole ID. We use the <code>spreadzone</code> operation, which has the following syntax:

<code>Result = spreadzone(points, initialfrictiondist, friction)</code>

Where:

* <code>points</code> is the raster with points to interpolate. In our case BoreholeID.
* <code>initialfrictiondist</code> is the initial friction distance. We don't need that, so we make it 0.
* <code>friction</code> is the friction for each cell, which we keep constant at 1.

In [16]:
ThiessenID = spreadzone(BoreholeID,0,1)

In [17]:
aguila(ThiessenID)

The final step is to assign the groundwater depth values of the boreholes to the Thiessen polygons.
We can do that by calculating the minimum or maximum depth per class, which will just extrapolate the value to the whole class. Here we'll use <code>areamaximum</code>:

In [18]:
BoreholeDepthThiessen = areamaximum(BoreholeDepth,ThiessenID)

Check the result and compare it with the IDW interpolation using Aguila.

In [19]:
aguila(BoreholeDepthIDW,BoreholeDepthThiessen)

### Assignments

-Write a script that calculates Thiessen polygons
-Make a function that can be applied to similar data as the `BoreholeDepth` raster.

### Acknowledgements

LinkedIn: [OkomoJacob](https://www.linkedin.com/in/okomo-jacob-622b86164/)<br>  
LinkedIn: [Hans van der Kwast](http://www.linkedin.com/in/jvdkwast)<br>

*__Twitter: [OkomoJacob](https://twitter.com/okomojacob)__*<br>